In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import pandas as pd
import numpy as np


In [2]:
import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

In [3]:

# a = pd.DataFrame(data, columns=['시분초','시가','대비','체결량','거래량'])

list = []
zero_df = pd.DataFrame(columns=['시분초','시가','대비','체결량','거래량'])
#list2 = []
#zero_df2 = pd.DataFrame(columns=['코드', '이름', '시간', '현재가', '대비', '시가', '고가', '저가', '매도호가', '매수호가', '거래량', '거래대금'])
        
class CpEvent:
    instance = None
 
    def OnReceived(self):
        # time = CpEvent.instance.GetHeaderValue(3)  # 시간
        timess = CpEvent.instance.GetHeaderValue(18)  # 초
        exFlag = CpEvent.instance.GetHeaderValue(19)  # 예상체결 플래그
        cprice = CpEvent.instance.GetHeaderValue(13)  # 현재가
        diff = CpEvent.instance.GetHeaderValue(2)  # 대비
        cVol = CpEvent.instance.GetHeaderValue(17)  # 순간체결수량
        vol = CpEvent.instance.GetHeaderValue(9)  # 거래량
 
        if (exFlag == ord('1')):  # 동시호가 시간 (예상체결)
            # print("실시간(예상체결)", timess, "*", cprice, "대비", diff, "체결량", cVol, "거래량", vol)
            list.append([timess,cprice,diff,cVol,vol])
            app_df = pd.DataFrame(data=list, columns=['시분초','시가','대비','체결량','거래량'])
            df = pd.concat([zero_df,app_df], ignore_index=True)
            print(df)
            JusikTimess = df['시분초'].to_list()
            JusikCprice = df['시가'].to_list()
            JusikDiff = df['대비'].to_list()
            JusikCVol = df['체결량'].to_list()
            JusikVol = df['거래량'].to_list()
            print(JusikTimess)
            for timess, cprice, diff, cvol, vol in zip(JusikTimess, JusikCprice, JusikDiff, JusikCVol, JusikVol):
                sql = "INSERT INTO jusik_tic_chart (JusikTimess, JusikCprice, JusikDiff, JusikCVol, JusikVol) VALUES (%s, %s, %s, %s, %s)" % ("'"+str(timess)+"'", "'"+str(cprice)+"'", "'"+str(diff)+"'", "'"+str(cvol)+"'","'"+str(vol)+"'")
                print(sql)
                cur.execute(sql)
            conn.commit()
           
            
        elif (exFlag == ord('2')):  # 장중(체결)
            #print("실시간(장중 체결)", timess, cprice, "대비", diff, "체결량", cVol, "거래량", vol)
            #print("실시간(예상체결)", timess, "*", cprice, "대비", diff, "체결량", cVol, "거래량", vol)
            list.append([timess,cprice,diff,cVol,vol])
            app_df = pd.DataFrame(data=list, columns=['시분초','시가','대비','체결량','거래량'])
            df = pd.concat([zero_df,app_df], ignore_index=True)
            print(df)
            JusikTimess = df['시분초'].to_list()
            JusikCprice = df['시가'].to_list()
            JusikDiff = df['대비'].to_list()
            JusikCVol = df['체결량'].to_list()
            JusikVol = df['거래량'].to_list()
            print(JusikTimess)
            for timess, cprice, diff, cvol, vol in zip(JusikTimess, JusikCprice, JusikDiff, JusikCVol, JusikVol):
                sql = "INSERT INTO jusik_tic_chart (JusikTimess, JusikCprice, JusikDiff, JusikCVol, JusikVol) VALUES (%s, %s, %s, %s, %s)" % ("'"+str(timess)+"'", "'"+str(cprice)+"'", "'"+str(diff)+"'", "'"+str(cvol)+"'","'"+str(vol)+"'")
                print(sql)
                cur.execute(sql)
            conn.commit()

class CpStockCur:
    def Subscribe(self, code):
        self.objStockCur = win32com.client.Dispatch("DsCbo1.StockCur")
        win32com.client.WithEvents(self.objStockCur, CpEvent)
        self.objStockCur.SetInputValue(0, 'U001')
        CpEvent.instance = self.objStockCur
        self.objStockCur.Subscribe()
 
    def Unsubscribe(self):
        self.objStockCur.Unsubscribe()
 
 
 
class CpStockMst:
    def Request(self, code):
        # 연결 여부 체크
        objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = objCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
            return False
 
        # 현재가 객체 구하기
        objStockMst = win32com.client.Dispatch("DsCbo1.StockIndexIS")
        objStockMst.SetInputValue(0,'U001')  # 종목 코드 - 한화에어로스페이스
        objStockMst.Subscribe()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = objStockMst.GetDibStatus()
        rqRet = objStockMst.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            return False
 
        # 현재가 정보 조회
        #code = objStockMst.GetHeaderValue(0)  # 종목코드
        time = objStockMst.GetHeaderValue(1)  # 종목명
        price = objStockMst.GetHeaderValue(2)  # 시간
        diff = objStockMst.GetHeaderValue(3)  # 종가
        volume = objStockMst.GetHeaderValue(4)  # 대비
        name = objStockMst.GetHeaderValue(6)  # 시가
        code = objStockMst.GetHeaderValue(7)  # 고가
        # low = objStockMst.GetHeaderValue(15)  # 저가
        # offer = objStockMst.GetHeaderValue(16)  # 매도호가
        # bid = objStockMst.GetHeaderValue(17)  # 매수호가
        # vol = objStockMst.GetHeaderValue(18)  # 거래량
        # vol_value = objStockMst.GetHeaderValue(19)  # 거래대금
 
        print("코드 이름 시간 시가 대비 거래량")
        print(code, name, time, price, diff, volume)
        #list2.append([code, name, time, cprice, diff, open, high, low, offer, bid, vol, vol_value])
        #app_df2 = pd.DataFrame(data=list2, columns=['시분초','시가','대비','체결량','거래량'])
        #df2 = pd.concat([zero_df2,app_df2], ignore_index=True)
        #print(df2)
        
        return True
 
 
class MyWindow(QMainWindow):
 
    def __init__(self):
        super().__init__()
        self.setWindowTitle("PLUS API TEST")
        self.setGeometry(300, 300, 300, 150)
        self.isRq = False
        self.objStockMst = CpStockMst()
        self.objStockCur = CpStockCur()
 
        btn1 = QPushButton("요청 시작", self)
        btn1.move(20, 20)
        btn1.clicked.connect(self.btn1_clicked)
 
        btn2 = QPushButton("요청 종료", self)
        btn2.move(20, 70)
        btn2.clicked.connect(self.btn2_clicked)
        # a = pd.DataFrame(data, columns=['시분초','시가','대비','체결량','거래량'])
 
        btn3 = QPushButton("종료", self)
        btn3.move(20, 120)
        btn3.clicked.connect(self.btn3_clicked)

 
    def StopSubscribe(self):
        if self.isRq:
            self.objStockCur.Unsubscribe()
        self.isRq = False
 
    def btn1_clicked(self):
        testCode = "KGG01P"
        if (self.objStockMst.Request(testCode) == False):
            exit()
 
        # 한화에어로스페이스 실시간 현재가 요청
        self.objStockCur.Subscribe(testCode)

    
        print("빼기빼기=====================================")
        print("실시간 현재가 요청 시작")
        self.isRq = True 
        
    #     JusikTimess = zero_df['시분초'].to_list()
    #     JusikCprice = zero_df['시가'].to_list()
    #     JusikDiff = zero_df['대비'].to_list()
    #     JusikCVol = zero_df['체결량'].to_list()
    #     JusikVol = zero_df['거래량'].to_list()
    #     print(JusikTimess)
    #     for timess, cprice, diff, cvol, vol in zip(JusikTimess, JusikCprice, JusikDiff, JusikCVol, JusikVol):
    #         sql = "INSERT INTO jusik_tic_chart (JusikTimess, JusikCprice, JusicDiff, JusikCVol, JusikVol) VALUES (%s, %s, %s, %s, %s)" % ("'"+timess+"'", "'"+cprice+"'", "'"+diff+"'", "'"+cvol+"'","'"+vol+"'")
    #         print(sql)
    #         cur.execute(sql)
    # conn.commit()


    def btn2_clicked(self):
        self.StopSubscribe()
 
 
    def btn3_clicked(self):
        self.StopSubscribe()
        exit()
        
 
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()


conn.close()

통신상태 0 
코드 이름 시간 시가 대비 거래량
rpm.rpm.clnr  1702047341 16684407.0 16348916.0 0
빼기빼기=====================================
실시간 현재가 요청 시작


In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 
 
################################################
# CpEvent: 실시간 이벤트 수신 클래스
class CpEvent:
    def set_params(self, client, name, caller):
        self.client = client  # CP 실시간 통신 object
        self.name = name  # 서비스가 다른 이벤트를 구분하기 위한 이름
        self.caller = caller  # callback 을 위해 보관
 
    def OnReceived(self):
        # 실시간 처리 - 현재가 주문 체결
        if self.name == 'stockcur':
            code = self.client.GetHeaderValue(0)  # 초
            name = self.client.GetHeaderValue(1)  # 초
            timess = self.client.GetHeaderValue(18)  # 초
            exFlag = self.client.GetHeaderValue(19)  # 예상체결 플래그
            cprice = self.client.GetHeaderValue(13)  # 현재가
            diff = self.client.GetHeaderValue(2)  # 대비
            cVol = self.client.GetHeaderValue(17)  # 순간체결수량
            vol = self.client.GetHeaderValue(9)  # 거래량
 
            if exFlag != ord('2'):
                return
 
            item = {}
            item['code'] = code
            item['time'] = timess
            item['diff'] = diff
            item['cur'] = cprice
            item['vol'] = vol
 
            # 현재가 업데이트
            self.caller.updateCurData(item)
 
 
            return
 
 
################################################
# plus 실시간 수신 base 클래스
class CpPublish:
    def __init__(self, name, serviceID):
        self.name = name
        self.obj = win32com.client.Dispatch(serviceID)
        self.bIsSB = False
 
    def Subscribe(self, var, caller):
        if self.bIsSB:
            self.Unsubscribe()
 
        if (len(var) > 0):
            self.obj.SetInputValue(0, var)
 
        handler = win32com.client.WithEvents(self.obj, CpEvent)
        handler.set_params(self.obj, self.name, caller)
        self.obj.Subscribe()
        self.bIsSB = True
 
    def Unsubscribe(self):
        if self.bIsSB:
            self.obj.Unsubscribe()
        self.bIsSB = False
 
 
################################################
# CpPBStockCur: 실시간 현재가 요청 클래스
class CpPBStockCur(CpPublish):
    def __init__(self):
        super().__init__('stockcur', 'DsCbo1.StockCur')
 
 
class CMinchartData:
    def __init__(self):
        self.minDatas = {}
        self.objCur = {}
 
    def stop(self):
        for k,v in self.objCur.items() :
            v.Unsubscribe()
 
 
    def addCode(self, code):
        if (code in self.minDatas) :
            return
 
        self.minDatas[code] = []
        self.objCur[code] = CpPBStockCur()
        self.objCur[code].Subscribe(code, self)
 
 
 
    def updateCurData(self, item):
        code = item['code']
        time  = item['time']
        cur = item['cur']
        self.makeMinchart(code, time, cur)
 
 
    def makeMinchart(self, code, time, cur) :
        hh, mm = divmod(time, 10000)
        mm, tt = divmod(mm, 100)
        mm += 1
        if (mm == 60) :
            hh += 1
            mm = 0
 
        hhmm = hh * 100 + mm
        if hhmm > 1530 :
            hhmm = 1530
        bFind = False
        minlen =len(self.minDatas[code])
        if (minlen > 0) :
            # 0 : 시간 1 : 시가 2: 고가 3: 저가 4: 종가
            if (self.minDatas[code][-1][0] == hhmm) :
                item = self.minDatas[code][-1]
                bFind = True
                item[4] = cur
                if (item[2] < cur):
                    item[2] = cur
                if (item[3] > cur):
                    item[3] = cur
 
        if bFind ==  False :
            self.minDatas[code].append([hhmm, cur, cur, cur, cur])
 
#        print(code, self.minDatas[code])
        return
 
    def print(self, code):
        print('====================================================-')
        print('분데이터 print', code, g_objCodeMgr.CodeToName(code))
        print('시간,시가,고가,저가,종가')
        for item in self.minDatas[code] :
            hh, mm = divmod(item[0], 100)
            print("%02d:%02d,%d,%d,%d,%d" %(hh, mm, item[1], item[2], item[3], item[4]))
 
 
 
# print(code, self.minDatas[code])
 
 
 
################################################
# 테스트를 위한 메인 화면
class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
 
        # plus 상태 체크
        if InitPlusCheck() == False:
            exit()
 
        self.minData = CMinchartData()
 
        # 코스피 200 종목 가져와 추가
        self.codelist = ['U001']
        for code in self.codelist :
            print(code, g_objCodeMgr.CodeToName(code))
            self.minData.addCode(code)
 
 
        self.setWindowTitle("주식 분 차트 생성")
        self.setGeometry(300, 300, 300, 180)
 
        nH  = 20
 
        btnPrint = QPushButton('print', self)
        btnPrint.move(20, nH)
        btnPrint.clicked.connect(self.btnPrint_clicked)
        nH += 50
 
 
        btnExit = QPushButton('종료', self)
        btnExit.move(20, nH)
        btnExit.clicked.connect(self.btnExit_clicked)
        nH += 50
 
 
    def btnPrint_clicked(self):
        for i in range(len(self.codelist)):
            self.minData.print(self.codelist[i])
            if i > 10: break
        return
 
 
    def btnExit_clicked(self):
        self.minData.stop()
        exit()
        return
 
 
 
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()

정상: 관리자권한으로 실행된 프로세스입니다.
U001 코스피
====================================================-
분데이터 print U001 코스피
시간,시가,고가,저가,종가
====================================================-
분데이터 print U001 코스피
시간,시가,고가,저가,종가
====================================================-
분데이터 print U001 코스피
시간,시가,고가,저가,종가
====================================================-
분데이터 print U001 코스피
시간,시가,고가,저가,종가
====================================================-
분데이터 print U001 코스피
시간,시가,고가,저가,종가
====================================================-
분데이터 print U001 코스피
시간,시가,고가,저가,종가
====================================================-
분데이터 print U001 코스피
시간,시가,고가,저가,종가


In [4]:
    # def real_jusik_tic():

    #     JusikTimess = zero_df['시분초'].to_list
    #     JusikCprice = zero_df['시가'].to_list
    #     JusikDiff = zero_df['대비'].to_list
    #     JusikCVol = zero_df['체결량'].to_list
    #     JusikVol = zero_df['거래량'].to_list

    #     for timess, cprice, diff, cvol, vol in zip(JusikTimess,JusikCprice,JusikDiff,JusikCVol,JusikVol):
    #         sql = "INSERT INTO jusik_tic_chart (JusikTimess,JusikCprice,JusikDiff,JusikCVol,JusikVol) VALUES (%s, %s, %s, %s, %s)" % ("'"+시분초+"'", "'"+시가+"'", "'"+대비+"'", "'"+체결량+"'", "'"+거래량+"'")
    #         print(sql)
    #         cur.execute(sql)

    #     conn.commit()